# Functional Evaluations
Haerang Lee

Find out a way to look into functional agreements.

In [1]:
from google.cloud import storage
import argparse
import gzip
import os
import sys
import time
from multiprocessing import Pool

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from utils import gcs_utils as gcs
from utils import model_and_evaluate_cluster as ev
import urllib.parse
import urllib.request

import io 

import importlib
import hdbscan
import networkx as nx 

In [29]:
importlib.reload(ev)

<module 'utils.model_and_evaluate_cluster' from '/Users/haeranglee/Documents/pss/utils/model_and_evaluate_cluster.py'>

# Tutorial

In [30]:
# Import possible pairs
prefix='model_outputs/no_cluster_size_limit/'
all_protein_combos_per_cluster = gcs.download_parquet(prefix+'B2-HDBSCAN-SeqVec-all_protein_combos_per_cluster.parquet')

In [31]:
funsim_result = ev.funsim_evaluator(all_protein_combos_per_cluster)

2021-Nov-07 15:53:28 No GO annotations provided. Downloading from google cloud.


/var/folders/zz/58vwrfdn6p5bgkhdv0xng5780000gn/T/ipykernel_27576/3331789406.py:1: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  funsim_result = ev.funsim_evaluator(all_protein_combos_per_cluster)


2021-Nov-07 15:53:31 Total number of proteins in GO annotations: 18240
2021-Nov-07 15:53:31 IC_t created
2021-Nov-07 15:53:32 Dictionary of proteins and their GO terms lookup created


In [32]:
funsim_result.funsim()
cluster_funsim, protein_pair_funsim = funsim_result.cluster_funsim, funsim_result.protein_pair_funsim

2021-Nov-07 15:53:34 Funsim calculated.
2021-Nov-07 15:53:34 Funsim summary by cluster done.
2021-Nov-07 15:53:34 Get NP Arr of GO terms for each protein
2021-Nov-07 15:53:34 Turn GO terms into dict
2021-Nov-07 15:53:35 Map GO desc...
2021-Nov-07 15:53:35 Mapping GO desc done.
2021-Nov-07 15:53:35 Common GO term sumary per cluster processed.
2021-Nov-07 15:53:35 Merged cluster-level funsim score with GO summary.


In [33]:
cluster_funsim.head()

,num_pairs,num_pairs_with_funsim,funsim,perc_pairs_w_funsim,cluster,go,go_summary
cluster,,,,,,,
0,45,45,0.477738,1.0,0,"{'GO:0001540': 1, 'GO:0004175': 2, 'GO:0004190...","{'GO:0004190': {'Num. Protein': 10, 'Name': 'a..."
1,276,0,NaN,0.0,1,{},{}
2,15,15,0.408240,1.0,2,"{'GO:0003723': 1, 'GO:0005198': 2, 'GO:0005509...","{'GO:0005509': {'Num. Protein': 5, 'Name': 'ca..."
3,10,10,0.724022,1.0,3,"{'GO:0003674': 1, 'GO:0004252': 5, 'GO:0005515...","{'GO:0004252': {'Num. Protein': 5, 'Name': 'se..."
4,15,15,0.407159,1.0,4,"{'GO:0003674': 3, 'GO:0005515': 4}","{'GO:0005515': {'Num. Protein': 4, 'Name': 'pr..."


In [34]:
protein_pair_funsim.head()

,protein_A,protein_B,cluster,funsim
1,O96009,P00797,0,0.428994
2,O96009,P07339,0,0.383279
3,O96009,P0DJD7,0,0.396631
4,O96009,P0DJD8,0,0.374980
5,O96009,P0DJD9,0,0.374980


Here's a function that will create a dataframe from GO summary of each cluster. In the below example, I pull the stats for cluster number 2. 

In [35]:
clusterno = 2
cluster_go_summary = funsim_result.get_go_summary_df(clusterno=clusterno)
cluster_go_summary

,Num. Protein,Name,Description
GO:0005509,5,calcium ion binding,Binding to a calcium ion (Ca2+).
GO:0046914,5,transition metal ion binding,Binding to a transition metal ions; a transiti...
GO:0005515,3,protein binding,Binding to a protein.
GO:0005198,2,structural molecule activity,The action of a molecule that contributes to t...
GO:0003723,1,RNA binding,Binding to an RNA molecule or a portion thereof.
GO:0030280,1,structural constituent of skin epidermis,The action of a molecule that contributes to t...
GO:0048306,1,calcium-dependent protein binding,Binding to a protein or protein complex in the...


# Background

## Gene Ontology

More info on Gene Ontology: http://geneontology.org/docs/ontology-documentation/
1. **Molecular function**: describe activities that occur at the molecular level, such as “catalysis” or “transport”. GO molecular function terms represent activities rather than the entities (molecules or complexes) that perform the actions
1. **Cellular component**: locations relative to cellular structures in which a gene product performs a function, either cellular compartments (e.g., mitochondrion), or stable macromolecular complexes of which they are parts (e.g., the ribosome)
1. **Biological process**: The larger processes, or ‘biological programs’ accomplished by multiple molecular activities. Examples of broad biological process terms are DNA repair or signal transduction.

## Functional Similarity Formula


Funcsim methodology from https://www.nature.com/articles/s41598-018-30455-0
> Functional similarity of a gene pair or a set is determined by the semantic similarities of the GO terms annotating the gene pair or set. Semantic similarity defines a distance between terms in the semantic space of GO and is quantified by the information contents (IC) of the terms. The information content (IC) of a GO term t is defined by negative log-likelihood:
$$IC(t)=-log(p(t))$$
> where term probability P(t) of term t is determined from the annotations of the corpus (corpus-based) or from the structure of the DAG (structure-based). The intuition is that terms in lower levels of DAG, that is, the terms with lower probability carry more specific information than the terms at higher levels in the hierarchy. Corpus-based methods evaluate the term probability as
$$p(t)= \frac{M}{N}$$
where M is the number of genes annotated by term t and N is the total number of genes in the annotating corpus.


## Data: Human Protein to GO Mapping

GAF: GO annotation files. 

Dataset `goa_human.gaf` downloaded from http://current.geneontology.org/products/pages/downloads.html
> **Filtered Files**
>
> These files are taxon-specific and reflect the work of specific projects, primarily the model organisms database groups, to provide comprehensive, non-redundant annotation files for their organism. All the files in this table have been filtered using the annotation file QC pipeline. A major component to the filtering is the requirement that particular taxon IDs can only be included within the association files provided by specific projects; the current list of authoritative groups and major model organisms can be found below. 

```
Homo sapiens
EBI Gene Ontology Annotation Database (goa) 	protein 	543477 	goa_human.gaf (gzip)
```
Data dictionary: http://geneontology.org/docs/go-annotation-file-gaf-format-2.1/




In [99]:
a_file = gzip. open("functional_sim/data/goa_human.gaf.gz", "rb")
contents = a_file. read()

In [178]:
print(contents.decode('utf-8')[0:1423])

!gaf-version: 2.2
!
!generated-by: GOC
!
!date-generated: 2021-10-27T15:09
!
!Header from source association file:
!=================================
!
!generated-by: GOC
!
!date-generated: 2021-10-27T04:08
!
!Header from goa_human source association file:
!=================================
!
!The set of protein accessions included in this file is based on UniProt reference proteomes, which provide one protein per gene.
!They include the protein sequences annotated in Swiss-Prot or the longest TrEMBL transcript if there is no Swiss-Prot record.
!If a particular protein accession is not annotated with GO, then it will not appear in this file.
!
!Note that the annotation set in this file is filtered in order to reduce redundancy; the full, unfiltered set can be found in
!ftp://ftp.ebi.ac.uk/pub/databases/GO/goa/UNIPROT/goa_uniprot_all.gz
!
!date-generated: 2021-06-16 11:28
!generated-by: UniProt
!go-version: http://purl.obolibrary.org/obo/go/releases/2021-06-06/extensions/go-plus.owl
!
!

In [135]:
goa = pd.read_csv("functional_sim/data/goa_human.gaf.gz", 
            compression='gzip', 
            header=None,
            skiprows=41, 
            sep='\t')
goa.columns=["DB",
                    "DB Object ID",
                    "DB Object Symbol",
                    "Qualifier",
                    "GO ID",
                    "Reference",
                    "Evidence Code",
                    "With or From",
                    "Aspect",
                    "Name",
                    "Synonym",
                    "Type",
                    "Taxon",
                    "Date",
                    "Assigned By",
                    "Annotation Extension",
                    "Gene Product Form ID"]
goa.head()

/Users/haeranglee/opt/anaconda3/envs/pss/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,DB,DB Object ID,DB Object Symbol,Qualifier,GO ID,Reference,Evidence Code,With or From,Aspect,Name,Synonym,Type,Taxon,Date,Assigned By,Annotation Extension,Gene Product Form ID
0,UniProtKB,A0A024RBG1,NUDT4B,enables,GO:0003723,GO_REF:0000043,IEA,UniProtKB-KW:KW-0694,F,Diphosphoinositol polyphosphate phosphohydrola...,NUDT4B,protein,taxon:9606,20210612,UniProt,NaN,NaN
1,UniProtKB,A0A024RBG1,NUDT4B,enables,GO:0046872,GO_REF:0000043,IEA,UniProtKB-KW:KW-0479,F,Diphosphoinositol polyphosphate phosphohydrola...,NUDT4B,protein,taxon:9606,20210612,UniProt,NaN,NaN
2,UniProtKB,A0A024RBG1,NUDT4B,enables,GO:0052840,GO_REF:0000003,IEA,EC:3.6.1.52,F,Diphosphoinositol polyphosphate phosphohydrola...,NUDT4B,protein,taxon:9606,20210612,UniProt,NaN,NaN
3,UniProtKB,A0A024RBG1,NUDT4B,enables,GO:0052842,GO_REF:0000003,IEA,EC:3.6.1.52,F,Diphosphoinositol polyphosphate phosphohydrola...,NUDT4B,protein,taxon:9606,20210612,UniProt,NaN,NaN
4,UniProtKB,A0A024RBG1,NUDT4B,located_in,GO:0005829,GO_REF:0000052,IDA,NaN,C,Diphosphoinositol polyphosphate phosphohydrola...,NUDT4B,protein,taxon:9606,20161204,HPA,NaN,NaN


In [164]:
goa.shape

(609748, 17)

In [149]:
goa["GO ID"].unique().size

18527

In [137]:
goa["DB"].unique()

array(['UniProtKB'], dtype=object)

The GOA human GAF file has 610K rows with protein IDs from UniProt.

In [140]:
goa["Qualifier"].unique()

array(['enables', 'located_in', 'involved_in', 'part_of', 'NOT|enables',
       'NOT|involved_in', 'is_active_in', 'NOT|colocalizes_with',
       'colocalizes_with', 'acts_upstream_of_or_within', 'contributes_to',
       'NOT|located_in', 'NOT|part_of', 'NOT|acts_upstream_of_or_within',
       'acts_upstream_of', 'acts_upstream_of_positive_effect',
       'acts_upstream_of_or_within_positive_effect',
       'acts_upstream_of_or_within_negative_effect', 'NOT|contributes_to',
       'acts_upstream_of_negative_effect',
       'NOT|acts_upstream_of_or_within_negative_effect',
       'NOT|is_active_in'], dtype=object)

In [184]:
len(goa["DB Object ID"].unique())

19788

In [185]:
# DB Object Symbol
len(goa["DB Object Symbol"].unique())

19718

There are about 20,000 unique proteins here. That's a great coverage. 

In [139]:
len(goa["GO ID"].unique())

18527

In [141]:
goa["Evidence Code"].unique()

array(['IEA', 'IDA', 'TAS', 'IPI', 'IEP', 'ISS', 'NAS', 'IMP', 'ISA',
       'HDA', 'EXP', 'ND', 'HEP', 'IC', 'RCA', 'HMP', 'IGI', 'IKR', 'IGC',
       'ISO', 'ISM', 'IBA'], dtype=object)

Taxonomy should be human. Some proteins may be found in multiple organisms other than human, but in the end, every data point in this dataset is in some way related to human.

In [183]:
goa["Taxon"].unique()[0:10]

array(['taxon:9606', 'taxon:9606|taxon:1280', 'taxon:9606|taxon:33892',
       'taxon:9606|taxon:11103', 'taxon:9606|taxon:11052',
       'taxon:9606|taxon:562', 'taxon:9606|taxon:197911',
       'taxon:9606|taxon:90370', 'taxon:9606|taxon:31649',
       'taxon:9606|taxon:1313'], dtype=object)

In [182]:
[taxon for taxon in goa["Taxon"].unique() if '9606' not in taxon]

[]

**QC** 

Can I find all GO in the human GOA dataset within GO BASIC?


I downloaded the GO Term hierarchy. The file I downloaded is `go-basic.obo` from http://geneontology.org/docs/download-ontology/ 

Description of the dataset from the source:
> This is the basic version of the GO, filtered such that the graph is guaranteed to be acyclic and annotations can be propagated up the graph. The relations included are is a, part of, regulates, negatively regulates and positively regulates. This version excludes relationships that cross the 3 GO hierarchies. This version should be used with most GO-based annotation tools.
go.obo and go.

In [ ]:
import obonet
import networkx as nx
gobasic = obonet.read_obo("functional_sim/data/go-basic.obo")

In [290]:
goa_goid_set = set(goa["GO ID"])
gobasic_set = set(gobasic.nodes)

In [291]:
goa_goid_set.difference(gobasic_set)

set()

GOA is a subset of gobasic, which is the full graph. 

In [292]:
len(gobasic_set.difference(goa_goid_set))

25323

I want to work only with GOMF. 

In [ ]:
r_gobasic = nx.reverse_view(gobasic_sub)

# 'GO:0003674' - This is the code for molecular function, which is the topmost parent
# I'm interested in all the nodes that are 1 distance away from the topmost parent. 


In [ ]:
# # Do not run again if already calculated. Import pickle file. 
# shortest_from_root = dict(nx.all_pairs_shortest_path_length(r_gobasic))
# with open('functional_sim/intermediary_data/shortest_from_root.pkl', 'wb') as file:
#     pickle.dump(shortest_from_root, file)

with open('functional_sim/intermediary_data/shortest_from_root.pkl', 'rb') as file:
    shortest_from_root = pickle.load(file)    
    

In [293]:
# How many GO's are in the shortest paths? 
len(shortest_from_root)

43850

In [294]:
# Here are how many eventually connect to our root, molecular_function.
len(shortest_from_root['GO:0003674'])

11168

There's a total of 11,168 GOMF terms in the GO-basic graph. In the human GAF dataset, there are only about 4,000. In addition to the species filter, the GAF dataset was filtered further through its QC checks. See [Gene Ontology wiki](http://wiki.geneontology.org/index.php/Release_Pipeline#Annotation_QC_checks) for more info.

In [295]:
# MF + BP + CC

len(shortest_from_root['GO:0003674']
   )+len(shortest_from_root['GO:0008150']
        )+len(shortest_from_root['GO:0005575'])

43850

43850  total GOs covered. These three sub-ontologies cover all the gene ontologies! And these are mutually exclusive categories, since, for this dataset, the cross-sub-ontology relationships have been removed.


In [193]:
# GOMF only 

goa_goid_mf = [goid for goid in set(goa["GO ID"]) if goid in shortest_from_root['GO:0003674'] ]
len(goa_goid_mf)

4431

In [194]:
goa_goid_mf[0:10]

['GO:0005412',
 'GO:0035254',
 'GO:0005035',
 'GO:0031690',
 'GO:0046980',
 'GO:1990247',
 'GO:0050567',
 'GO:0052630',
 'GO:0052814',
 'GO:0003943']

# Data: GO ID, Name, and Desc



But I also want the go term names.

From http://geneontology.org/docs/faq/

>    How do I get the term names for my list of GO ids?
>
>    You can use the YeastMine Analyze tool available at SGD to retrieve the GO term names for each ID.
>
>    Go to the Analyze tool on YeastMine
>    In the Select Type pull down, select GO Term
>    Enter your GO ids or upload a list in the full format (GO:0016020, GO:0016301…)
>    Click on Create List. The tool offers several options to download the list.



In [ ]:
pd.DataFrame(goa_goid_mf).to_csv('functional_sim/goa_goid_mf.csv', index=False, header=False)

I uploaded `goa_goid_mf.csv` onto YeastMine and downloaded `yeastmine_results_goa_goid_mf.tsv`.

In [13]:
go_term_names = pd.read_csv('functional_sim/data/yeastmine_results_goa_goid_mf.tsv', sep='\t')
go_term_names.columns = [col[10:] for col in go_term_names.columns]
go_term_names.shape

(4431, 4)

In [14]:
go_term_names.head()

,Identifier,Name,Namespace,Description
0,GO:0000009,"alpha-1,6-mannosyltransferase activity",molecular_function,Catalysis of the transfer of a mannose residue...
1,GO:0000010,trans-hexaprenyltranstransferase activity,molecular_function,Catalysis of the reaction: all-trans-hexapreny...
2,GO:0000014,single-stranded DNA endodeoxyribonuclease acti...,molecular_function,Catalysis of the hydrolysis of ester linkages ...
3,GO:0000016,lactase activity,molecular_function,Catalysis of the reaction: lactose + H2O = D-g...
4,GO:0000026,"alpha-1,2-mannosyltransferase activity",molecular_function,Catalysis of the transfer of a mannose residue...
